<a href="https://colab.research.google.com/github/gopalm-ai/Quantum_Machine_Learning_An_Applied_Approach/blob/main/HHL_qiskit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# HHL Algorithm

In [2]:
! pip3 install qiskit

     |████████████████████████████████| 6.4 MB 14.5 MB/s 
     |████████████████████████████████| 18.0 MB 439 kB/s 
     |████████████████████████████████| 238 kB 39.8 MB/s 
     |████████████████████████████████| 200 kB 32.1 MB/s 
     |████████████████████████████████| 53 kB 1.5 MB/s 
     |████████████████████████████████| 1.6 MB 32.9 MB/s 
     |████████████████████████████████| 38.2 MB 1.4 MB/s 
     |████████████████████████████████| 49 kB 4.9 MB/s 
     |████████████████████████████████| 49 kB 3.8 MB/s 
     |████████████████████████████████| 38.1 MB 1.3 MB/s 
     |████████████████████████████████| 943 kB 44.1 MB/s 
     |████████████████████████████████| 3.6 MB 35.3 MB/s 
     |████████████████████████████████| 112 kB 41.0 MB/s 
  Created wheel for qiskit: filename=qiskit-0.34.1-py3-none-any.whl size=11771 sha256=016ac1b32228265c4edc85779ac6f85347eb2e40866125ecb9c741629e81d269
  Stored in directory: /root/.cache/pip/wheels/79/b1/3f/8cdfd5543a84705e4bd16e081f2362b9b3bfd9898d2e2

In [3]:
import numpy as np
from qiskit import *
pi = np.pi
q = QuantumRegister(6)
c = ClassicalRegister(2)
hhl = QuantumCircuit(q, c)

In [4]:
# Superposition and Quantum Circuit Definition

In [6]:
# Superposition
hhl.h(q[1])
hhl.h(q[2])
# Controlled-U0
hhl.cu3(-pi/2, -pi/2, pi/2, q[2], q[3])
# hhl.cu1(3*pi/4, q[2], q[3])
hhl.cp(3*pi/4, q[2], q[3])
hhl.cx(q[2], q[3])
# hhl.cu1(3*pi/4, q[2], q[3])
hhl.cp(3*pi/4, q[2], q[3])
hhl.cx(q[2], q[3])
# Controlled-U1
hhl.cx(q[1], q[3])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: The QuantumCircuit.cu3 method is deprecated as of 0.16.0. It will be removed no earlier than 3 months after the release date. You should use the QuantumCircuit.cu method instead, where cu3(ϴ,φ,λ) = cu(ϴ,φ,λ,0).
  """


In [7]:
# Inverse QFT

In [8]:
hhl.swap(q[1], q[2])
hhl.h(q[2])
hhl.cp(-pi/2, q[1], q[2])
hhl.h(q[1])

In [9]:
# Ancilla Rotation 

In [13]:
hhl.cu(0.392699, 0, 0, 0, q[1], q[0]) # Controlled-RY0
hhl.cu(0.19634955, 0, 0, 0, q[2], q[0]) # Controlled-RY1

In [14]:
# QPE Uncompute Action

In [15]:
hhl.swap(q[1], q[2])
hhl.h(q[1])
hhl.cp(pi/2, q[1], q[2]) # Inverse(Dagger(Controlled-S))
hhl.h(q[2])
hhl.swap(q[2], q[1])
# Inverse(Controlled-U1)
hhl.cx(q[1], q[3])
# Inverse(Controlled-U0)
hhl.cx(q[2], q[3])
hhl.cp(-3*pi/4, q[2], q[3])
hhl.cx(q[2], q[3])
hhl.cp(-3*pi/4, q[2], q[3])
hhl.cu(-pi/2, pi/2, -pi/2, 0, q[2], q[3])
# End of Inverse(controlled-U0)
hhl.h(q[2])
hhl.h(q[1])

In [16]:
# Post-selection

In [19]:
# Target state preparation
hhl.rz(-pi, q[4])
hhl.p(pi, q[4])
hhl.h(q[4])
hhl.ry(-0.9311623288419387, q[4])
hhl.rz(pi, q[4])
# Swap test
hhl.h(q[5])
hhl.cx(q[4], q[3])
hhl.ccx(q[5], q[3], q[4])
hhl.cx(q[4], q[3])
hhl.h(q[5])
hhl.barrier(q)
hhl.measure(q[0], c[0])
hhl.measure(q[5], c[1])

In [20]:
# Success Functions

In [21]:
def get_psuccess(counts):

  try:
    succ_rotation_fail_swap = counts['11']
  except KeyError:
    succ_rotation_fail_swap = 0
  try:
    succ_rotation_succ_swap = counts['01']
  except KeyError:
    succ_rotation_succ_swap = 0
  succ_rotation = succ_rotation_succ_swap + succ_rotation_fail_swap
  try:
    prob_swap_test_success = succ_rotation_succ_swap / succ_rotation
  except ZeroDivisionError:
    prob_swap_test_success = 0
  return prob_swap_test_success

In [22]:
# Run circuit

In [23]:
backend = BasicAer.get_backend('qasm_simulator')
job = execute(hhl, backend, shots=100)
result = job.result()
counts = result.get_counts(hhl)
print(get_psuccess(counts))

1.0
